
# Homework 2. LDA for classification
Jorge Alejandro Díaz Sánchez


The description of this task in PDF can be downloaded at the following link: Homework description file

Links to an external site.

In this exercise, the dataset that will be used is “Heart_Disease”. This database contains 76 attributes, but all published experiments refer to using a subset of 14 of them.  In particular, the Cleveland database is the only one that has been used by ML researchers to date.  The "goal" field refers to the presence of heart disease in the patient. It is integer valued from 0 (no presence) to 4. Experiments with the Cleveland database have concentrated on simply attempting to distinguish presence (values 1,2,3,4) from absence (value 0). The dataset can be obtained in the repository “UC Irvine Machine Learning Repository” (https://archive.ics.uci.edu/dataset/45/heart+disease
Links to an external site. ). For this exercise, you can use any of the files namely “processed.xxxxx.data”. You should consider that there are some differences between the classes in some of those files.

In [1]:
random_seed = 0

In [2]:
import sys
sys.path.append(r"~/Documents/GitHub/PredictiveModelling/")
import ax_utilities as utl
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [3]:
df = pd.read_csv("Data/heart_disease/processed.cleveland.data", header=None, names=['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','num'], na_values=["?"])
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45.0,1.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0,1
299,68.0,1.0,4.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0,2
300,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0,3
301,57.0,0.0,2.0,130.0,236.0,0.0,2.0,174.0,0.0,0.0,2.0,1.0,3.0,1


Attribute Information:
   -- Only 14 used
      -- 1. #3  (age)
      -- 2. #4  (sex)      
      -- 3. #9  (cp)        
      -- 4. #10 (trestbps)  
      -- 5. #12 (chol)      
      -- 6. #16 (fbs)       
      -- 7. #19 (restecg)   
      -- 8. #32 (thalach)   
      -- 9. #38 (exang)     
      -- 10. #40 (oldpeak)   
      -- 11. #41 (slope)     
      -- 12. #44 (ca)        
      -- 13. #51 (thal)      
      -- 14. #58 (num)       (the predicted attribute)

In [4]:
utl.data_quality_report(df)

,0,Type,Missing_values,Present_values,Unique_values,Min_value,Max_value
age,age,float64,0,303,41,29.0,77.0
sex,sex,float64,0,303,2,0.0,1.0
cp,cp,float64,0,303,4,1.0,4.0
trestbps,trestbps,float64,0,303,50,94.0,200.0
chol,chol,float64,0,303,152,126.0,564.0
fbs,fbs,float64,0,303,2,0.0,1.0
restecg,restecg,float64,0,303,3,0.0,2.0
thalach,thalach,float64,0,303,91,71.0,202.0
exang,exang,float64,0,303,2,0.0,1.0
oldpeak,oldpeak,float64,0,303,40,0.0,6.2


There are some non numeric  characters "?" in variables "ca" and "thal", we are replacing them with np.nan, to drop them.

In [5]:
utl.data_quality_report(df)

,0,Type,Missing_values,Present_values,Unique_values,Min_value,Max_value
age,age,float64,0,303,41,29.0,77.0
sex,sex,float64,0,303,2,0.0,1.0
cp,cp,float64,0,303,4,1.0,4.0
trestbps,trestbps,float64,0,303,50,94.0,200.0
chol,chol,float64,0,303,152,126.0,564.0
fbs,fbs,float64,0,303,2,0.0,1.0
restecg,restecg,float64,0,303,3,0.0,2.0
thalach,thalach,float64,0,303,91,71.0,202.0
exang,exang,float64,0,303,2,0.0,1.0
oldpeak,oldpeak,float64,0,303,40,0.0,6.2


In [6]:
df.isna().sum().sum() / df.shape[0]

0.019801980198019802

In [7]:
df.dropna(inplace=True)

In [8]:
inputs = df.drop(columns="num")
inputs.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal'],
      dtype='object')

In [9]:
target = df["num"]
target.value_counts()

0    160
1     54
2     35
3     35
4     13
Name: num, dtype: int64

In [10]:
# we are only interested if there's a hearth disease (1-4) or not (0), so we are grouping 1-4 into just 1
target.iloc[target != 0] = 1

/var/folders/51/l057c8xx2md597vhk47n57rm0000gn/T/ipykernel_71887/2469113571.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target.iloc[target != 0] = 1


In [11]:
target.value_counts()

0    160
1    137
Name: num, dtype: int64

## Test and Train subsets

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(inputs, target, random_state=random_seed)

## Useful Utility

In [14]:
from sklearn.metrics import (precision_score, accuracy_score, recall_score)

def get_model_metrics(actual, predicted):
    print("Accuracy: ", accuracy_score(actual, predicted))
    print("Recall: ", recall_score(actual, predicted))
    print("Precision: ", precision_score(actual, predicted))

## Base Logistic Regression Model
Train a logistic regression model to estimate the feature “target” using all the pixels in the
image as input variables. Get the accuracy values or another metric (precision, recall, …) to
evaluate the model's performance in training and testing.

In [15]:
from sklearn.linear_model import LogisticRegression

In [16]:
base_logistic_model = LogisticRegression()
base_logistic_model.fit(X_train, y_train)
y_predicted_train = base_logistic_model.predict(X_train)
y_predicted_test = base_logistic_model.predict(X_test)

/Users/adiaz/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
get_model_metrics(y_train, y_predicted_train)

Accuracy:  0.8603603603603603
Recall:  0.7722772277227723
Precision:  0.9069767441860465


In [18]:
get_model_metrics(y_test, y_predicted_test)

Accuracy:  0.84
Recall:  0.75
Precision:  0.9


In [19]:
base_logistic_model.coef_.T

array([[-0.0400846 ],
       [ 0.55619431],
       [ 0.45733375],
       [ 0.02277421],
       [ 0.00258287],
       [-0.27483743],
       [ 0.2088982 ],
       [-0.0436271 ],
       [ 0.85102295],
       [ 0.27910511],
       [ 0.1489019 ],
       [ 0.96456194],
       [ 0.35538362]])

In [20]:
base_logistic_model.intercept_

array([-0.19576745])

## PCA + Logistic Regression
Considering the data set obtained in point 1, perform the dimension reduction using the
PCA method. With the variables resulting from the reduction process, train a new logistic
regression model and calculate the same metrics used in point 2.

In [21]:
# For this case, let's first transform the data using Principal Component Analysis
from sklearn.decomposition import PCA

In [22]:
pca_transformer = PCA()
X_train_pca = pca_transformer.fit_transform(X_train)
pca_transformer.explained_variance_ratio_

array([7.41054037e-01, 1.49414651e-01, 9.08156749e-02, 1.64738904e-02,
       1.05399797e-03, 3.54037728e-04, 2.47375626e-04, 2.22757762e-04,
       1.78577690e-04, 6.25303642e-05, 4.91249405e-05, 3.89292783e-05,
       3.44156868e-05])

In [23]:
from numpy import cumsum

cumsum(pca_transformer.explained_variance_ratio_)

array([0.74105404, 0.89046869, 0.98128436, 0.99775825, 0.99881225,
       0.99916629, 0.99941366, 0.99963642, 0.999815  , 0.99987753,
       0.99992666, 0.99996558, 1.        ])

In [24]:
pca_transformer.components_

array([[ 3.75847743e-02, -1.43387682e-03,  1.24359574e-04,
         6.10652659e-02,  9.97413189e-01,  1.35850514e-04,
         3.16749703e-03, -1.89617264e-03,  4.01062860e-04,
         1.84440301e-03, -2.69053556e-04,  2.31927005e-03,
         7.84523861e-04],
       [ 2.01361719e-01,  3.77910789e-04,  1.40814641e-02,
         1.13768101e-01, -1.64884919e-02,  5.21531336e-04,
         3.53390005e-03, -9.72083801e-01,  8.37345287e-03,
         1.57971214e-02,  9.83034305e-03,  1.15521956e-02,
         2.30336033e-02],
       [ 9.46891734e-02, -1.28967005e-03, -6.68379930e-03,
         9.83956931e-01, -6.36029980e-02,  2.80608811e-03,
         7.60307572e-03,  1.36170471e-01, -8.98027424e-04,
         8.95654952e-03,  2.59808317e-03,  2.00420439e-03,
         8.43622478e-03],
       [-9.73586690e-01,  7.13843167e-03,  5.04386359e-03,
         1.21625461e-01,  2.89515946e-02, -5.58807700e-03,
         1.17815333e-03, -1.87962633e-01,  8.21188758e-03,
         2.53679309e-03,  4.84114559e

In [25]:
import pickle

# open a file to save PCA Matrix
file = open('pca_matrix.pickle', 'wb')

# dump information to that file
pickle.dump(pca_transformer.components_, file)

# close the file
file.close()

In [26]:
X_train_pca = X_train_pca[:,:3]

In [27]:
X_train_pca

array([[-5.45967104e+01, -1.02300487e+01, -2.87674875e+01],
       [-8.32605439e+01,  1.11276895e+01,  3.14690835e+01],
       [-1.00801334e+02,  2.69517236e+01, -1.67767730e+01],
       [ 4.40491722e+01, -1.80794967e+01,  6.74936683e+00],
       [ 7.85754618e+01,  2.15928539e+00,  4.22064635e+01],
       [ 1.90858217e+01, -1.43833176e+01, -3.62352046e+00],
       [ 3.34374592e+01,  4.52738148e+01, -1.19522704e+01],
       [ 8.27975342e+00,  9.00360591e+00, -9.72139760e+00],
       [-4.11120579e+01, -4.60036747e+01, -9.13018351e+00],
       [-9.08348433e+01,  8.84441778e+00, -1.81690712e+01],
       [-4.46729488e+01,  3.88887147e+01, -2.62939795e+01],
       [-4.83696667e+01, -7.34611869e+00,  4.26177367e+00],
       [ 8.03740015e+01, -1.87982100e+01, -5.53480428e+00],
       [ 5.54449217e+01, -1.70875209e+01, -1.06568049e+01],
       [ 2.34392210e+01,  1.07155327e+01,  4.21491589e+01],
       [ 2.18131446e+01,  4.29885939e+01,  2.10077834e+01],
       [-6.51855182e+01,  4.51756094e+01

In [28]:
pca_logistic_model = LogisticRegression()
pca_logistic_model.fit(X_train, y_train)
y_predicted_train = pca_logistic_model.predict(X_train)
y_predicted_test = pca_logistic_model.predict(X_test)

/Users/adiaz/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [29]:
get_model_metrics(y_train, y_predicted_train)

Accuracy:  0.8603603603603603
Recall:  0.7722772277227723
Precision:  0.9069767441860465


In [30]:
get_model_metrics(y_test, y_predicted_test)

Accuracy:  0.84
Recall:  0.75
Precision:  0.9


## LDA Transform + Logistic Regression
Considering the data set used in point 1 again, perform a variable reduction by LDA
transformation. With the variables resulting from the transformation process, train a new
logistic regression wich must be evaluated with the same metrics as the previous models.

In [31]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [32]:
lda_transformer = LinearDiscriminantAnalysis()
X_train_lda = lda_transformer.fit_transform(X_train, y_train)

In [33]:
X_train_lda

array([[-1.63338027],
       [ 2.56791727],
       [-1.01683219],
       [ 1.53537198],
       [ 0.23703633],
       [-1.39540269],
       [ 1.89837885],
       [ 2.03420849],
       [-2.75728741],
       [-2.17723169],
       [ 0.87997755],
       [-1.245     ],
       [-1.11357106],
       [-2.21889363],
       [ 0.71925636],
       [ 2.24938042],
       [ 2.07045138],
       [ 1.47088487],
       [ 1.90301143],
       [-1.51927273],
       [-1.48834167],
       [ 0.72388963],
       [-1.52822716],
       [ 1.59919094],
       [ 0.71068113],
       [ 1.15294237],
       [-0.58111627],
       [-0.40083531],
       [ 0.44837476],
       [ 1.70010565],
       [-2.38350667],
       [ 0.97121028],
       [ 0.29603848],
       [ 1.4368981 ],
       [ 0.06509877],
       [ 0.87995643],
       [ 2.30391068],
       [ 2.41051496],
       [ 0.59135777],
       [-2.30835706],
       [ 1.39164386],
       [-2.14182911],
       [-0.68845023],
       [ 0.35848889],
       [-0.77538621],
       [-1

In [34]:
lda_transformer.coef_

array([[ 0.00392773,  1.00891307,  0.8882975 ,  0.02934885,  0.00404914,
        -0.68755987,  0.22614151, -0.02312235,  1.55861919,  0.22953944,
         0.49811978,  0.93889697,  0.4836753 ]])

In [35]:
# open a file to save LDA coefficients
file = open('lda_coef.pickle', 'wb')

# dump information to that file
pickle.dump(lda_transformer.coef_, file)

# close the file
file.close()

In [36]:
lda_logistic_model = LogisticRegression()
lda_logistic_model.fit(X_train, y_train)
y_predicted_train = lda_logistic_model.predict(X_train)
y_predicted_test = lda_logistic_model.predict(X_test)

/Users/adiaz/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [37]:
get_model_metrics(y_train, y_predicted_train)

Accuracy:  0.8603603603603603
Recall:  0.7722772277227723
Precision:  0.9069767441860465


In [38]:
get_model_metrics(y_test, y_predicted_test)

Accuracy:  0.84
Recall:  0.75
Precision:  0.9


## LDA MODEL
Considering the data set used in point 1 again; train a LDA model and evaluate the model
with the same metrics

In [39]:
lda_model = LinearDiscriminantAnalysis()
lda_model.fit(X_train, y_train)
y_predicted_train = lda_model.predict(X_train)
y_predicted_test = lda_model.predict(X_test)

In [40]:
get_model_metrics(y_train, y_predicted_train)

Accuracy:  0.8558558558558559
Recall:  0.8118811881188119
Precision:  0.8631578947368421


In [41]:
get_model_metrics(y_test, y_predicted_test)

Accuracy:  0.8533333333333334
Recall:  0.75
Precision:  0.9310344827586207


In [44]:
lda_model.coef_.T

array([[ 0.00392773],
       [ 1.00891307],
       [ 0.8882975 ],
       [ 0.02934885],
       [ 0.00404914],
       [-0.68755987],
       [ 0.22614151],
       [-0.02312235],
       [ 1.55861919],
       [ 0.22953944],
       [ 0.49811978],
       [ 0.93889697],
       [ 0.4836753 ]])

In [43]:
lda_model.intercept_

array([-10.22924791])